## Imports

In [1]:
import requests
import pandas as pd
import re
import numpy as np
import random
import glob
import os

from bs4 import BeautifulSoup
from IPython.display import clear_output

from itertools import cycle
import urllib.request
from urllib3 import ProxyManager, make_headers

from tqdm import tqdm
from time import sleep

## Proxy Stuff

In [2]:
def get_proxy_header():
    Hosts = [
        ('us.proxymesh.com', 31280),
        ('us-fl.proxymesh.com', 31280),
        ('us-wa.proxymesh.com', 31280),
        #('fr.proxymesh.com', 31280),
        #('jp.proxymesh.com', 31280)
    ]

    ip_address = Hosts[random.randint(0,2)]

    host = ip_address[0]
    port = ip_address[1]
    return 'http://{}:{}'.format(host,port)

def getUserAgent():
    UAS = (
       "Mozilla/5.0 (Linux; Android 10; VOG-L29) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.105 Mobile Safari/537.36 OPR/62.3.3146.57763",
       "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 OPR/75.0.3969.149",
       "Mozilla/5.0 (Windows NT 10.0; WOW64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 OPR/75.0.3969.149",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 OPR/74.0.3911.23",
       'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 YaBrowser/21.3.0 Yowser/2.5 Safari/537.36',
        )
    return UAS[random.randrange(len(UAS))]

def getProxyManager():
    default_headers = make_headers(proxy_basic_auth='trd_haru:IHavePassword@0@0311')
    http = ProxyManager(get_proxy_header(), proxy_headers=default_headers)
    return http

def checkProxy(http):
    r = http.request('GET', 'http://httpbin.org/ip')
    print(r.data)

In [12]:
http = getProxyManager()

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 OPR/74.0.3911.23',
           'scheme': 'https'}

## Read in

In [3]:
filelist = glob.glob('*.csv')

In [4]:
frames = [pd.read_csv(f) for f in filelist]

In [5]:
df = pd.concat(frames)

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [8]:
df = df.drop(columns=['NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME'])

In [9]:
df['PROPERTY TYPE'].value_counts()

Condo/Co-op                  159
Single Family Residential    150
Vacant Land                   14
Townhouse                      7
Multi-Family (2-4 Unit)        2
Name: PROPERTY TYPE, dtype: int64

GET via scrape: 

1. Original listing price
2. % Change from original to list price
3. Sale price
4. % Change from original to sold price
5. Days on market

In [10]:
print(df['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/140-S-Hibiscus-Dr-33139/home/42789091


In [15]:
URL = df['URL'].iloc[0]

In [16]:
r = http.request('GET', URL, headers=headers)
soup = BeautifulSoup(r.data, 'html.parser')

In [31]:
timeline = soup.find?

In [32]:
timeline = soup.find('div', {'class': 'timeline-content'})

In [35]:
soup

<!DOCTYPE html>
<html lang="en"><head>
<!-- OptanonConsentNoticeStart -->
<!-- OneTrust Cookies Consent Notice start for redfin.com -->
<script charset="UTF-8" data-domain-script="7e5bc3d6-ef20-4760-aa0d-c8df4649fae2" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript"></script>
<script type="text/javascript">
				function OptanonWrapper() {
				}
			</script>
<!-- OptanonConsentNoticeEnd -->
<!-- IAB CCPA US Privacy String Configuration-->
<script ccpa-opt-out-geo="us" ccpa-opt-out-ids="C0004" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript"></script>
<!-- Server: phantom-23 --><!-- Time generated: Wed Apr 12 2023 09:37:03 GMT-0700 (Pacific Daylight Time) --><script>(function(a){window.__reactServerOnClickHandler=function(i){(a[i]=a[i]||[]).push(new Date)}})(window.__reactServerUnhandledEvents={})</script><script>
/*! LAB.js (LABjs :: Loading And Blocking JavaScript)
    v2.0.3 (c) Kyl